# Auto-Encoder Example

Build a 2 layers auto-encoder with TensorFlow to compress images to a lower latent space and then reconstruct them.

- Author: Aymeric Damien
- Project: https://github.com/aymericdamien/TensorFlow-Examples/

## Auto-Encoder Overview

<img src="http://kvfrans.com/content/images/2016/08/autoenc.jpg" alt="ae" style="width: 800px;"/>

References:
- [Gradient-based learning applied to document recognition](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf). Y. LeCun, L. Bottou, Y. Bengio, and P. Haffner. Proceedings of the IEEE, 86(11):2278-2324, November 1998.

## MNIST Dataset Overview

This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

More info: http://yann.lecun.com/exdb/mnist/

In [0]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# Import MNIST data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")

In [0]:
# Graph Clear
tf.reset_default_graph()

In [0]:
# Training Parameters
learning_rate = 0.01

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, 784])

In [0]:
# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    we1 = tf.Variable(tf.random_normal([784, 256]))
    be1 = tf.Variable(tf.random_normal([256]))
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, we1), be1))

    # Encoder Hidden layer with sigmoid activation #2
    we2 = tf.Variable(tf.random_normal([256, 128]))
    be2 = tf.Variable(tf.random_normal([128]))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, we2), be2))

    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    wd1 = tf.Variable(tf.random_normal([128, 256]))
    bd1 = tf.Variable(tf.random_normal([256]))
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, wd1), bd1))
    
    # Decoder Hidden layer with sigmoid activation #2
    wd2 = tf.Variable(tf.random_normal([256, 784]))
    bd2 = tf.Variable(tf.random_normal([784]))
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, wd2), bd2))

    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.pow(X - decoder_op, 2))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)


In [0]:
epochs = 20
batch_size = 64
display_step = int(5000 / batch_size)
num_batches = int(mnist.train.num_examples / batch_size)

# Start Training
# Start a new TF session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for e in range(epochs):
    print('epoch %d' % (e + 1))
    
    # Training
    for i in range(num_batches):
        # Prepare Data
        # Get the next batch of MNIST data (only images are needed, not labels)
        batch_x, _ = mnist.train.next_batch(batch_size)

        # Run optimization op (backprop) and cost op (to get loss value)
        sess.run(optimizer, feed_dict={X: batch_x})
        # Display logs per step
        if ((i + 1) % display_step == 0):
            l = sess.run(loss, feed_dict={X: batch_x})
            print('Step %i: Minibatch Loss: %f' % ((i + 1) * batch_size, l))
            

l = sess.run(loss, feed_dict={X: mnist.test.images})
print('Test set Loss: %f' % l)

In [0]:
# Testing
# Encode and decode images from test set and visualize their reconstruction.
n = 4
canvas_orig = np.empty((28 * n, 28 * n))
canvas_recon = np.empty((28 * n, 28 * n))
for i in range(n):
    # MNIST test set
    batch_x, _ = mnist.test.next_batch(n)
    # Encode and decode the digit image
    g = sess.run(decoder_op, feed_dict={X: batch_x})
    
    # Display original images
    for j in range(n):
        # Draw the generated digits
        canvas_orig[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = batch_x[j].reshape([28, 28])
    # Display reconstructed images
    for j in range(n):
        # Draw the generated digits
        canvas_recon[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = g[j].reshape([28, 28])

print("Original Images")     
plt.figure(figsize=(n, n))
plt.imshow(canvas_orig, origin="upper", cmap="gray")
plt.show()

print("Reconstructed Images")
plt.figure(figsize=(n, n))
plt.imshow(canvas_recon, origin="upper", cmap="gray")
plt.show()

### 모델에 fashion mnist를 넣고 돌리면?

In [0]:
'''
트레이닝 데이터로 활용할 60000개 이미지, 테스트용 10000개 이미지
28*28 사이즈의 이미지가 그냥 주욱 784칸 짜리 어레이에 담겨있다.
test, traing 데이터가 분리되어 있다.
레이블은 one-hot encoding을 해주어야 한다. 
'''

(train_x, train_y), (test_x, test_y) = tf.keras.datasets.fashion_mnist.load_data()

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [0]:
# Testing fashion
# Encode and decode images from test set and visualize their reconstruction.
n = 4
canvas_orig = np.empty((28 * n, 28 * n))
canvas_recon = np.empty((28 * n, 28 * n))
for i in range(n):
    # MNIST test set
    batch_x = train_x[:n].reshape(-1, 784)
    # Encode and decode the digit image
    g = sess.run(decoder_op, feed_dict={X: batch_x})
    
    # Display original images
    for j in range(n):
        # Draw the generated digits
        canvas_orig[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = batch_x[j].reshape([28, 28])
    # Display reconstructed images
    for j in range(n):
        # Draw the generated digits
        canvas_recon[i * 28:(i + 1) * 28, j * 28:(j + 1) * 28] = g[j].reshape([28, 28])

print("Original Images")     
plt.figure(figsize=(n, n))
plt.imshow(canvas_orig, origin="upper", cmap="gray")
plt.show()

print("Reconstructed Images")
plt.figure(figsize=(n, n))
plt.imshow(canvas_recon, origin="upper", cmap="gray")
plt.show()